In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
I_GPU = 2

In [2]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.multimodal.s3dis import S3DISFusedDataset
from torch_points3d.datasets.segmentation.multimodal import IGNORE_LABEL

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

# Load config

In [6]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/s3dis'  # ENGIE

overrides = [
    'task=segmentation',
#     'data=segmentation/multimodal/s3disfused/no3d_5cm_768x384-exact_no-pixel-height',
#     'data=segmentation/multimodal/s3disfused/3d_2d/sparse/no_pixel_height/5cm_1024x512-exact',
    'data=segmentation/multimodal/s3disfused/3d_2d/sparse/no_pixel_height/5cm_512x256-exact',
#     'models=segmentation/multimodal/no3d',
    'models=segmentation/multimodal/sparseconv3d',
#     'model_name=RGB_ResNet18PPM_mean-feat',
    'model_name=Res16UNet34-L4-early-ade20k-interpolate',
    
    'data.fold=5',
    'data.sample_per_epoch=2000',
#     f"data.dataroot={os.path.join(DATA_ROOT, '5cm_exact_768x384')}",
    f"data.dataroot={os.path.join(DATA_ROOT, '5cm_exact_512x256')}",
]

cfg = hydra_read(overrides)

# print(OmegaConf.to_yaml(cfg))

# Load S3DIS dataset

In [4]:
from time import time
start = time()
    
dataset = S3DISFusedDataset(cfg.data)
# print(dataset)

print(f"Time = {time() - start:0.1f} sec.")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Time = 97.1 sec.


In [ ]:
# torch.save(dataset, '/home/ign.fr/drobert-admin/Bureau/s3dis_test_bckp.pt')

In [ ]:
# from time import time
# start = time()

# dataset = torch.load('/home/ign.fr/drobert-admin/Bureau/s3dis_test_bckp.pt')

# print(f"Time = {time() - start:0.1f} sec.")

# Load model

In [ ]:
from torch_points3d.models.model_factory import instantiate_model

# print(f"\nInitial GPU state")
init_memory = torch.cuda.memory_allocated(0)
print(f"CUDA initial memory : {init_memory / (1024*1024):0.1f} Mo")

print(f"\nModel initialization")
print(f"Model: {cfg.model_name}")
model = instantiate_model(cfg, dataset)
model = model.train().cuda()
# model = model.train().cpu()
model_memory = torch.cuda.memory_allocated(0) - init_memory
print(f"CUDA model memory : {model_memory / (1024*1024):0.1f} Mo")
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model parameters : {n_params / 10**6:0.1f} M")

print(f"\nData loading")
from torch_points3d.core.multimodal.data import MMBatch
# batch = MMBatch.from_mm_data_list([dataset.train_dataset[0], dataset.train_dataset[0], dataset.train_dataset[0], dataset.train_dataset[0]])
batch = MMBatch.from_mm_data_list([dataset.train_dataset[0]])
if model.is_multimodal:
#     n_pixels = sum([im.num_views * im.img_size[0] * im.img_size[1] for im in batch.modalities['image']])
#     credit_max = dataset.train_transform_image.transforms[-1].credit
    print(batch)
#     print(f"Used pixel credit: {n_pixels} / {credit_max} = {n_pixels / credit_max * 100:0.1f} %")
else:
    batch = batch.data
    print(batch)

print(f"\nForward pass")
model.set_input(batch, model.device)
batch = model(batch)
forward_memory = torch.cuda.memory_allocated(0) - init_memory - model_memory
print(f"CUDA activations memory : {forward_memory / (1024*1024):0.1f} Mo")
#if model.is_multimodal:
#    print(f"Memory cost per pixel : {forward_memory / 1024 / n_pixels:0.1f} ko/pixel")

print(f"\nLoss")
model.loss_seg

print(f"\nBackward pass")
model.backward()

print(f"\nOK")

del model, batch

# Dataset iteration speed

In [ ]:
from time import time

n_iter = 100
start = time()
for _ in range(n_iter):
    _ = dataset.train_dataset[0]
time_per_iter = (time() - start) / n_iter
print(f"Time per iteration: {time_per_iter:0.4f}")

# Model feature distribution on dataset